# Naive Simple MLP

In [ ]:
# ================= QUIET HEADER (경고 최소화; 맨 위에 두기) =================
import sys, types, warnings
try:
    import tqdm as _tqdm
    _auto = types.ModuleType("tqdm.auto"); _auto.tqdm = _tqdm.tqdm
    sys.modules["tqdm.auto"] = _auto
except Exception:
    pass
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning
warnings.filterwarnings("ignore", category=DataConversionWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", message=".*IProgress not found.*")
warnings.filterwarnings("ignore", message=".*Maximum iterations.*hasn't converged.*")
# ===========================================================================

# ================= USER CONFIG =============================================
DATA_FILE     = "dataset.csv"               # index='Time'
Y_COLS        = ["xr_2","xr_3","xr_5","xr_7","xr_10"]

SLOPE_PREFIX  = "s_"                        # 예: s_2, s_3, ...
MACRO_PREFIX  = "F"                         # 예: FIP, FUMD, F... 등 'F'로 시작
FWD_PREFIX    = "fwd_"                      # 예: fwd_1, fwd_2, ...

PERIOD        = ["197108", "202312"]
BURN_IN_END   = "199001"
HORIZON       = 12
SHOW_PROGRESS = True

RUN_RIDGE     = True
RUN_MLP       = True

# 자주 바꿀만한 최소 파라미터/그리드만 노출
RIDGE_PARAMS  = dict(random_state=0)        # alpha는 그리드에서 튜닝
RIDGE_CV      = {"mode":"tscv","n_splits":10,"grid":{"alpha":[1e-3,1e-2,1e-1,1,10,100]}}

MLP_PARAMS    = dict(                        # 고정 기본값(필요시만 수정)
    random_state=0,
    max_iter=2000,
    early_stopping=True,
    learning_rate_init=1e-3,
    tol=1e-5,
)
MLP_CV        = {"mode":"tscv","n_splits":8,
                 "grid":{"hidden_layer_sizes":[(16,), (16, 8)], "alpha":[1e3, 1e5, 1e7]}}
# ===========================================================================

import os
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from rolling_framework import ExpandingRunner, make_strategy

# ----------------------- helpers -----------------------
def read_df(path: str) -> pd.DataFrame:
    df = pd.read_csv(path, index_col="Time"); df.index = df.index.astype(str)
    return df

def cols_by_prefix(df: pd.DataFrame, prefix: str) -> pd.DataFrame:
    return df.loc[:, df.columns.str.startswith(prefix)]

def make_features(df: pd.DataFrame, *, use_slope: bool, use_macro: bool, use_fwd: bool) -> pd.DataFrame:
    parts = []
    if use_slope: parts.append(cols_by_prefix(df, SLOPE_PREFIX))
    if use_macro: parts.append(cols_by_prefix(df, MACRO_PREFIX))
    if use_fwd:   parts.append(cols_by_prefix(df, FWD_PREFIX))
    if not parts: raise ValueError("No features selected.")
    X = pd.concat(parts, axis=1)
    return X.loc[:, ~X.columns.duplicated(keep="first")]  # 중복 제거

def slope_map_from_targets(df: pd.DataFrame, ycols, slope_prefix: str):
    suffix     = [c.split("_", 1)[1] for c in ycols]
    slope_cols = [f"{slope_prefix}{s}" for s in suffix]
    missing = [c for c in slope_cols if c not in df.columns]
    if missing:
        raise KeyError(f"Missing slope columns: {missing}")
    return dict(zip(ycols, slope_cols))

def build_cs_baseline(runner: ExpandingRunner, df: pd.DataFrame, y: pd.DataFrame,
                      slope_map: dict) -> pd.DataFrame:
    """y_cshat: 각 만기 j에서 xr_j ~ s_j OLS, expanding-OOS 예측"""
    rows = []
    for t in runner.test_times:
        tr = [s for s in runner.times if s < t]
        if not tr: 
            continue
        row = {}
        for ycol, scol in slope_map.items():
            X_tr = df.loc[tr, [scol]].astype(float)             # DF로 fit
            y_tr = y.loc[tr, ycol].astype(float).values
            reg  = LinearRegression().fit(X_tr, y_tr)
            x_te = pd.DataFrame([[df.loc[t, scol]]], columns=[scol], dtype=float)  # DF로 predict
            row[ycol] = float(reg.predict(x_te))
        rows.append(pd.Series(row, name=t))
    return pd.DataFrame(rows).reindex(index=runner.test_times, columns=list(slope_map.keys()))

def run_experiment(model_name: str, params: dict, cv: dict,
                   X: pd.DataFrame, y: pd.DataFrame, tag: str, scale: bool = True):
    strat  = make_strategy(model_name, target_cols=Y_COLS, params=params, scale=scale, cv=cv)
    runner = ExpandingRunner(X=X, y=y, strategy=strat,
                             period=PERIOD, burn_in_end=BURN_IN_END, horizon=HORIZON)
    runner.fit_walk(progress=SHOW_PROGRESS, desc=f"{model_name} on {tag}")

    # Custom benchmark: y_cshat (CS OLS baseline)
    s_map    = slope_map_from_targets(df, Y_COLS, SLOPE_PREFIX)
    y_cshat  = build_cs_baseline(runner, df, y, s_map)

    r2_naive = runner.R2OOS(baseline="naive").round(4)
    r2_cond  = runner.R2OOS(baseline="condmean").round(4)
    r2_cs    = runner.R2OOS(baseline="custom", benchmark=y_cshat).round(4)

    print(f"\n=== {model_name} on {tag} ===")
    print("R2OOS vs naive:\n",     r2_naive)
    print("R2OOS vs condmean:\n",  r2_cond)
    print("R2OOS vs CS OLS(y_cshat):\n", r2_cs)

    # 필요하면 저장 (주석 해제)
    # runner.to_mat(f"{model_name.lower()}_{tag.replace(' ','_').lower()}.mat",
    #               baseline="custom", benchmark=y_cshat)
    # y_cshat.to_csv(f"ycshat_{tag.replace(' ','_').lower()}.csv")

    return runner, y_cshat

# ----------------------- load & split -------------------
df = read_df(DATA_FILE)
y  = df[Y_COLS].copy()

# 두 실험용 특징세트
X_slope_fwd        = make_features(df, use_slope=True, use_macro=False, use_fwd=True)
X_slope_macro_fwd  = make_features(df, use_slope=True, use_macro=True,  use_fwd=True)

# ----------------------- RUN: Ridge ---------------------
if RUN_RIDGE:
    _runner, _ycshat = run_experiment("Ridge", RIDGE_PARAMS, RIDGE_CV, X_slope_fwd,       y, "Slope + Fwd",           scale=True)
    _runner, _ycshat = run_experiment("Ridge", RIDGE_PARAMS, RIDGE_CV, X_slope_macro_fwd, y, "Slope + Macro + Fwd",   scale=True)

# ----------------------- RUN: MLP -----------------------
if RUN_MLP:
    _runner, _ycshat = run_experiment("MLP", MLP_PARAMS, MLP_CV, X_slope_fwd,       y, "Slope + Fwd",         scale=True)
    _runner, _ycshat = run_experiment("MLP", MLP_PARAMS, MLP_CV, X_slope_macro_fwd, y, "Slope + Macro + Fwd", scale=True)